In [1]:
import tensorflow as tf
from tensorflow.python.platform import build_info as build
print(f"tensorflow version: {tf.__version__}")
print(f"Cuda Version: {build.build_info['cuda_version']}")
print(f"Cudnn version: {build.build_info['cudnn_version']}")

2024-06-15 22:13:39.384166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 22:13:39.384273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 22:13:39.385148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 22:13:39.391045: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 22:13:40.127549: W tensorflow/compiler/tf2

tensorflow version: 2.15.0
Cuda Version: 12.2
Cudnn version: 8


In [2]:
import tensorflow as tf
import tf2onnx
import onnx

In [3]:
model = tf.keras.models.load_model('test.keras')

2024-06-15 22:13:48.361917: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:13:48.381460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:13:48.381505: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:13:48.383522: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:13:48.383580: I external/local_xla/xla/stream_executor

In [4]:
if isinstance(model.input, list):
    input_details = [{'shape': input_tensor.shape, 'dtype': input_tensor.dtype, 'name': input_tensor.name} for input_tensor in model.inputs]
else:
    input_details = [{'shape': model.input.shape, 'dtype': model.input.dtype, 'name': model.input.name}]

# Print input details
for i, detail in enumerate(input_details):
    print(f"Input {i+1}: shape: {detail['shape']}, dtype: {detail['dtype']}, name: {detail['name']}")


Input 1: shape: (None, 807), dtype: <dtype: 'float32'>, name: review
Input 2: shape: (None, 1), dtype: <dtype: 'float32'>, name: types


In [6]:
input_signature = [
    tf.TensorSpec(shape=(None, 807), dtype=tf.float32, name='review'),
    tf.TensorSpec(shape=(None, 1), dtype=tf.float32, name='types')
]

In [8]:
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=18)
onnx.save(onnx_model, 'test.onnx')

2024-06-15 22:16:03.212237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:16:03.212286: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-06-15 22:16:03.212396: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-06-15 22:16:03.212729: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:16:03.212795: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-15 22:16:03.212834: I external/local_xla/xl

In [ ]:
# !python -m tf2onnx.convert --saved-model saved-model --output model.onnx

In [9]:
!pip show onnxruntime-gpu

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Name: onnxruntime-gpu
Version: 1.18.0
Summary: ONNX Runtime is a runtime accelerator for Machine Learning models
Home-page: https://onnxruntime.ai
Author: Microsoft Corporation
Author-email: onnxruntime@microsoft.com
License: MIT License
Location: /home/martinus/anaconda3/envs/testing/lib/python3.11/site-packages
Requires: coloredlogs, flatbuffers, numpy, packaging, protobuf, sympy
Required-by: 


In [11]:
import onnxruntime as ort

session = ort.InferenceSession('test.onnx', providers=['CUDAExecutionProvider'])

2024-06-15 22:17:11.150884051 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-06-15 22:17:11.150915601 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [28]:
import numpy as np

# Dummy input data (replace with real data)
review_input = np.random.rand(1, 807).astype(np.float32)
types_input = np.random.rand(1, 1).astype(np.float32)

# Prepare inputs
inputs = {
    "review": review_input,
    "types": types_input
}

results_ort = session.run(None, inputs)

print(results_ort)

[array([[2.9289556]], dtype=float32)]


In [31]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# from keras_tuner import RandomSearch, HyperModel

# Load the dataset
file_path = 'combined-dataset/final_reviews_data.csv'
data = pd.read_csv(file_path)

# Encode the 'types' column
label_encoder = LabelEncoder()
data['types_encoded'] = label_encoder.fit_transform(data['types'])

# Tokenize the 'review' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad the sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Create the feature set
X = {
    'review': padded_sequences.astype(np.float32),
    'types': data['types_encoded'].values.astype(np.float32),
}

# Normalize the sentiment scores
y = data['sentiment'].values


In [32]:
df_review = pd.read_csv('combined-dataset/final_reviews_data.csv')
df_place = pd.read_csv('combined-dataset/combined_datasetV2.csv')

In [34]:
import numpy as np

# Dummy input data (replace with real data)
review_input = X['review']
types_input = X['types'].reshape(-1, 1)

# Prepare inputs
inputs = {
    "review": review_input,
    "types": types_input
}

results_ort = session.run(None, inputs)

print(results_ort)

2024-06-15 22:34:54.303951393 [E:onnxruntime:, sequential_executor.cc:516 ExecuteKernel] Non-zero status code returned while running Gather node. Name:'model_1/embedding_2/embedding_lookup' Status Message: /onnxruntime_src/onnxruntime/core/providers/cuda/shared_inc/fast_divmod.h:50 onnxruntime::cuda::DivMod<int>::DivMod(int) d_ >= 1 && d_ <= static_cast<uint32_t>(std::numeric_limits<int>::max()) was false. 



RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Gather node. Name:'model_1/embedding_2/embedding_lookup' Status Message: /onnxruntime_src/onnxruntime/core/providers/cuda/shared_inc/fast_divmod.h:50 onnxruntime::cuda::DivMod<int>::DivMod(int) d_ >= 1 && d_ <= static_cast<uint32_t>(std::numeric_limits<int>::max()) was false. 
